In [2]:
import torch

In [4]:
probs = torch.load('outputs_torch.pt').cpu()

In [4]:
import torch 
d = torch.load('data_batches/train06r076_37.pt')
print(d.keys())
samples, ref_stft, target, perceived_signals, doas = d['samples'], d['ref_stft'], d['target'], d['perceived_signals'], d['doas']
print(f"samples.shape={samples.shape}")
print(f"ref_stft.shape={ref_stft.shape}")
print(f"target.shape={target.shape}")
print(f"perceived_signals.shape={perceived_signals.shape}")
print(f"doas.shape={doas.shape}")

dict_keys(['samples', 'ref_stft', 'target', 'perceived_signals', 'doas'])
samples.shape=torch.Size([64, 14, 256, 96])
ref_stft.shape=torch.Size([64, 257, 96])
target.shape=torch.Size([64, 256, 96])
perceived_signals.shape=torch.Size([64, 2, 12160])
doas.shape=torch.Size([64, 2])


In [25]:
d1 = {'samples': samples[:2], 'ref_stft': ref_stft[:2], 'target': target[:2], 'mixed_signals': perceived_signals[:2], 'doas': doas[:2]}
print([v.shape for k,v in d1.items()])
torch.save(d1, 'example_batch2.pt')

[torch.Size([2, 14, 256, 1120]), torch.Size([2, 257, 1120]), torch.Size([2, 256, 1120]), torch.Size([2, 2, 149119]), torch.Size([2, 2])]


In [18]:

angles_idx = torch.argmax(probs, dim=1)
vals = angles_idx[0].unique(return_counts=True)

probs.shape=torch.Size([8, 13, 256, 1120])


In [19]:
print(vals[1].sort())

torch.return_types.sort(
values=tensor([ 1063,  2157,  4026,  5348,  7552, 11676, 13289, 14329, 31553, 37984,
        50508, 53235, 54000]),
indices=tensor([ 9, 10, 11,  7,  2,  5,  3,  0,  1,  8, 12,  4,  6]))


In [9]:
import bsseval

class SeparatedSource:
	"""Represents the part of the received sound that comes from this
	specific source (angle).
	"""
	def __init__(self, ref_spec, probs):
		# The ref mic's spectrogram, shape=(Bmw, t)
		self.ref_spec = ref_spec
		# The model's output for this angle, shape=(w, t)
		self.probs = probs
	
	def energy(self):
		return torch.sum(self.probs * abs(self.ref_spec) ** 2)
	
	def spec(self):
		mag = abs(self.ref_spec) * self.probs
		phase = torch.angle(self.ref_spec)
		return mag * torch.exp(1j * phase)
	
	def metrics(self):
		"""Retuns SDR, ISR, SIR, SAR."""
		sep_signal = torch.istft(self.spec(), ...)  # <-- Finish me
		# TODO: maybe use the reference time signal directly?
		ref_signal = torch.istft(self.ref_spec, ...)  # <-- Finish me
		
		return bsseval.evaluate(
			references=sep_signal.reshape(...),  # <-- Finish me
			estimates=ref_signal.reshape(...),  # <-- Finish me
			# win=1*44100,
			# hop=1*44100,
			# mode='v4',
			# padding=True
		)
	
	@classmethod
	def speaker_angles(cls, sources):
		"""Returns the 2 angle numbers where the speakers are (most
		likely) located.
		"""
		energies = [src.energy() for src in sources]
		# `argpartition` to get the indices, AKA angle numbers.
		partitioned = np.argpartition(energies, angle_count - 2)
		max_angles = tuple(partitioned[-2:])
		return max_angles
	
	@classmethod
	def sample_metrics(cls, ref_spec, samp_probs):
		"""Retuns the metrics for this sample's data & label."""
		# `samp_probs`'s shape is (angle_count, w, t).
		# `sources[i]` is the separeted source coming from the direction
		# theta_i.
		sources = [SeparatedSource(ref_spec, probs) for probs in samp_probs]
		
		speaker_angles = SeparatedSource.speaker_angles(sources)
		speaker_metrics = [
			sources[angle.spec].metrics()
			for angle in speaker_angles
		]
		return speaker_metrics

# Loop for each sample in the batch
# for ref_spec, samp_probs in zip(batch_ref_specs, batch_outputs):
	# print(SeparatedSource.sample_metrics(ref_spec, samp_probs))


In [15]:
ref_stft[1:].shape

torch.Size([256, 1120])

In [21]:
probs.view(list(probs.shape)[:-2] + [-1]) @ torch.abs(ref_stft)
torch.sum(probs * abs(ref_stft[1:]) ** 2)

tensor(3090761., grad_fn=<SumBackward0>)

In [ ]:
SeparatedSource(target, ref)

In [1]:
from audio_proj_generator_v2 import TrainLibriSpeechAudio

In [2]:
gen = TrainLibriSpeechAudio()
df = gen.generate_dataset_meta()
display(df)

,speaker_name,record_name,audio_path,pair_id,reverb_time,SIR,theta,radius
0,3000,15664,source_signals/LibriSpeech/dev-clean\3000\1566...,0,0.4,-0.532481,180,1.185416
1,6345,64257,source_signals/LibriSpeech/dev-clean\6345\6425...,0,0.2,1.280003,135,2.390319
2,5694,64029,source_signals/LibriSpeech/dev-clean\5694\6402...,1,0.3,1.999142,60,1.786532
3,8842,302203,source_signals/LibriSpeech/dev-clean\8842\3022...,1,0.4,-0.769222,150,1.344866
4,1462,170142,source_signals/LibriSpeech/dev-clean\1462\1701...,2,0.4,1.778789,120,1.955771
...,...,...,...,...,...,...,...,...
2043,8297,275154,source_signals/LibriSpeech/dev-clean\8297\2751...,1021,0.2,0.338972,30,1.362223
2044,2078,142845,source_signals/LibriSpeech/dev-clean\2078\1428...,1022,0.2,1.326748,135,1.566237
2045,6295,244435,source_signals/LibriSpeech/dev-clean\6295\2444...,1022,0.2,-1.349233,30,1.677237
2046,6241,61943,source_signals/LibriSpeech/dev-clean\6241\6194...,1023,0.4,0.940246,150,1.701565


In [32]:
from torchaudio.transforms import Spectrogram, InverseSpectrogram
import torchaudio

FRAME_LENGTH = 512 # K
NFFT = FRAME_LENGTH  # FFT length in STFT
OVERLAP = 0.75  # Frame overlap in STFT
HOP_LENGTH = int((1 - OVERLAP) * NFFT)  # Hop length in STFTHOP_LENGTH = 
FS = 16e3

waveform, sample_rate = torchaudio.load('source_signals/LibriSpeech/train-clean-100/27/124992/27-124992-0001.flac')
waveform = waveform.cuda()
print(f"waveform.shape={waveform.shape}")

spectrogram = Spectrogram(
    n_fft=NFFT,
    win_length=NFFT,
    hop_length=HOP_LENGTH,
    power=None  # Complex spectrum
).to('cuda')


ispectrogram = InverseSpectrogram(
    n_fft=NFFT,
    win_length=NFFT,
    hop_length=HOP_LENGTH,
).to('cuda')



waveform.shape=torch.Size([1, 222560])


In [56]:
def print_len(waveform, len):
    wfl = waveform[:, :len]
    stft = spectrogram(wfl)
    istft = ispectrogram(stft)
    print(f"original shape={wfl.shape}")
    print(f"stft shape={stft.shape}")
    print(f"istft(stft) shape={istft.shape}")

In [65]:
print_len(waveform, int(16000*10.235))
1271//16 * 16
16000*10.235

original shape=torch.Size([1, 163760])
stft shape=torch.Size([1, 257, 1280])
istft(stft) shape=torch.Size([1, 163712])


163760.0

In [47]:
print_len(waveform, 12220)

original shape=torch.Size([1, 12220])
stft shape=torch.Size([1, 257, 96])
istft(stft) shape=torch.Size([1, 12160])


In [60]:
from audio_dataset import DictTorchPartedDataset
import torch 
from tqdm import tqdm

device='cuda'
train_dataset = DictTorchPartedDataset('data_batches', 'train06r076' , ['samples', 'ref_stft', 'target'], real_batch_size=64, virtual_batch_size=1, device=device)

for i in tqdm(range(4864, len(train_dataset))):
    samples, _, _ = train_dataset[i]
    if torch.any(torch.isnan(samples)):
        print(f"{i} has {len(torch.isnan(samples) / samples.numel() * 100)}% nans")

100%|██████████| 1152/1152 [01:24<00:00, 13.56it/s]


In [2]:
print(f"{i} : _{i // 64}.pt, member {i % 64}")

data = torch.load('data_batches/train06r076_75.pt')
d = {k:v[30] for k,v in data.items()}

NameError: name 'torch' is not defined

In [38]:
samples, ref_stft, target, perceived_signals, doas = d['samples'], d['ref_stft'], d['target'], d['perceived_signals'], d['doas']
print(f"samples.shape={samples.shape}")
print(f"ref_stft.shape={ref_stft.shape}")
print(f"target.shape={target.shape}")
print(f"perceived_signals.shape={perceived_signals.shape}")
print(f"doas.shape={doas.shape}")

samples.shape=torch.Size([14, 256, 96])
ref_stft.shape=torch.Size([257, 96])
target.shape=torch.Size([256, 96])
perceived_signals.shape=torch.Size([2, 12160])
doas.shape=torch.Size([2])


In [53]:
len(perceived_signals[1][torch.isnan(perceived_signals[1])])

0

In [15]:
import torch
data = torch.load('data_batches/__SSS__test10tgtprc_0.pt')
d = {k: torch.tensor(v[:2].detach().cpu().numpy()) for k,v in data.items()}
d['probs'] = torch.tensor(probs.detach().cpu().numpy())

In [16]:
torch.save(d, 'samples_test1405.pt')

In [1]:
import scipy
mat_path = 'timit_rir/Impulse_response_Acoustic_Lab_Bar-Ilan_University_(Reverberation_0.160s)_3-3-3-8-3-3-3_1m_000.mat'
mat = scipy.io.loadmat(mat_path)

In [23]:
rir = mat['impulse_response']

In [56]:
import torch
import torchaudio
a = torch.tensor(rir.T, dtype=torch.float32)
print(f"a.shape={a.shape}")

a.shape=torch.Size([8, 480000])


C:\Users\agadi\AppData\Local\Temp\ipykernel_15444\196252497.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  a = torch.tensor(rir.T, dtype=torch.float32)


In [57]:
rir = a.flip(dims=(0, ))
print(rir.shape)
waveform = torchaudio.transforms.Resample(48000, 16000)(rir)

torch.Size([8, 480000])


In [60]:
def _load_mat_as_torch(mat_path: str) -> torch.tensor:
    mat = scipy.io.loadmat(mat_path)
    data_impulse_response = mat['impulse_response']
    impulse_response_fs = 48000

    rir = torch.tensor(data_impulse_response.T, dtype=torch.float32)
    # Reorder microphones
    if True:
        rir = rir.flip((0, ))
        
    # 8 x XXX
    waveform = torchaudio.transforms.Resample(impulse_response_fs, 16000)(rir)

    return waveform

In [61]:
wav1 = _load_mat_as_torch('timit_rir/Impulse_response_Acoustic_Lab_Bar-Ilan_University_(Reverberation_0.160s)_3-3-3-8-3-3-3_1m_000.mat')
print(wav1.shape)

torch.Size([8, 160000])
